In [ ]:
import numpy as np
import limp

# Conference scheduling

Given a bunch of sessions and rooms to host them in, assign time slots to minimize scheduling of sessions with mutual interest at the same time.
Adding more time slots seems to increase complexity faster than adding more rooms.
Beyond 4-5 time slots, exact minimization becomes very expensive.

Minimization is slow, but constraint satisfaction is almost instant.
Thus, we can set a time limit for optimization depending on how long we're willing to wait for a better answer.
Empirically, the estimated lower bounds are over-optimistic, and the solutions we find are actually quite good compared to optimal.

## Room, Session, Time
A more explicit formulation that maps sessions to specific rooms.
Doesn't seem to scale beyond ~20 sessions.

In [ ]:
time_limit_sec = 30
rng = np.random.default_rng(12345)

# room_sizes = [200, 100, 50, 50]
room_sizes = [300, 200, 100, 50, 50, 25]
N_rooms = len(room_sizes)
N_times = 4
N_sessions = N_rooms * N_times

# Number of people wanting to attend each session
X = rng.exponential(size=N_sessions)
sess_interest = (X * (max(room_sizes) / X.max())).round().astype('int')

# Number of people interested in attending both sessions,
# who will be frustrated if they're scheduled at the same time.
X = np.triu(  rng.exponential(size=(N_sessions, N_sessions)),  k=1)
X = X + X.T # symmetric
sess_pairs = (X * (max(sess_interest) / X.max())).round().astype('int')

In [ ]:
p = limp.Problem()
empty_seats = limp.Expr()

# Is session s held in room r at time t?
V = np.empty((N_rooms, N_sessions, N_times), dtype=object)
for r in range(N_rooms):
    for s in range(N_sessions):
        for t in range(N_times):
            V[r,s,t] = v = p.binvar(f's{s}_r{r}_t{t}')
            if sess_interest[s] > room_sizes[r]:
                p.equal(v, 0) # session won't fit in this room
            else:
                # Total interest and total available seats are both fixed, so minimizing the difference does nothing
                # But we can minimize the square of empty seats, which will put the biggest sessions in the biggest rooms
                empty_seats += v * (room_sizes[r] - sess_interest[s])**2

# Each session may be held exactly once
for s in range(N_sessions):
    p.exactly(1, V[:,s,:].flatten())

# For a given room and time, at most one session may be scheduled
for r in range(N_rooms):
    for t in range(N_times):
        p.at_most(1, V[r,:,t].flatten())

conflicts = limp.Expr()
for s1 in range(N_sessions):
    for s2 in range(s1+1, N_sessions):
        for t in range(N_times):
            # We know v1 and v2 have bounds 0,1
            # s1 is scheduled at time t?
            v1 = p.binvar(f's{s1}_t{t}'); p.equal(v1, p.sum(V[:,s1,t]))
            # s2 is scheduled at time t?
            v2 = p.binvar(f's{s2}_t{t}'); p.equal(v2, p.sum(V[:,s2,t]))
            # s1 and s2 are scheduled at the same time?
            v12 = p.prod(v1, v2)
            conflicts += v12 * sess_pairs[s1,s2]

# ans = p.minimize(0) # no objective, just find an answer
# ans = p.minimize(empty_seats) # biggest sessions in the biggest rooms
# %time ans = p.minimize(conflicts, time_limit=time_limit_sec) # so everyone can see everything
%time ans = p.minimize(conflicts + 1e-4*empty_seats, time_limit=time_limit_sec) # both criteria
# 10 sec for 4*4 (1700 v, 3000 c), 30 sec for 4*5, 3.5 min for 4*6 (3900 v, 6900 c)

rr, ss, tt = np.vectorize(lambda x: ans['by_var'][x])(V).round().nonzero()
for t, r, s in sorted(zip(tt, rr, ss)):
    print(f'Time {t}    Room {r}    Session {s:2d}   Demand {sess_interest[s]:3d} of {room_sizes[r]:3d}')
print(f'Objective:    {ans['soln'].fun}')
print(f'# variables:  {len(ans['vs'])}')
print(f'# constraints:  {len(ans['constraints'].A)}')

## Session and Time only
Simpler formulation that doesn't assign rooms, just times.
Somewhat faster at large problem sizes and scales better.

Scales to at least 6 times x 10 rooms (60 sessions).  Finds better solutions after 120 sec than after 30 sec, but 300 sec only offers 20% additional improvement.

In [ ]:
time_limit_sec = 30
rng = np.random.default_rng(12345)

# room_sizes = [200, 100, 50, 50]
# room_sizes = [300, 200, 100, 50, 50]
# room_sizes = [300, 200, 100, 50, 50, 25]
room_sizes = [300, 200, 100, 100, 50, 50, 25, 25, 25, 25]
N_rooms = len(room_sizes)
N_times = 6
N_sessions = N_rooms * N_times

# Number of people wanting to attend each session
X = rng.exponential(size=N_sessions)
sess_interest = (X * (max(room_sizes) / X.max())).round().astype('int')

# Number of people interested in attending both sessions,
# who will be frustrated if they're scheduled at the same time.
X = np.triu(  rng.exponential(size=(N_sessions, N_sessions)),  k=1)
X = X + X.T # symmetric
sess_pairs = (X * (max(sess_interest) / X.max())).round().astype('int')

In [ ]:
p = limp.Problem()

# Is session s held at time t?
V = np.empty((N_sessions, N_times), dtype=object)
for s in range(N_sessions):
    for t in range(N_times):
        V[s,t] = v = p.binvar(f's{s}_t{t}')

# Each session may be held exactly once
for s in range(N_sessions):
    p.exactly(1, V[s,:])

# At time t, at most N_rooms sessions may be scheduled
for t in range(N_times):
    p.at_most(N_rooms, V[:,t])

# Don't schedule sessions at the same time if people want to attend both
conflicts = limp.Expr()
for s1 in range(N_sessions):
    for s2 in range(s1+1, N_sessions):
        for t in range(N_times):
            # s1 and s2 are scheduled at the same time?
            v12 = p.prod(V[s1,t], V[s2,t])
            conflicts += v12 * sess_pairs[s1,s2]

# If we schedule all the big sessions at the same time, we could run out of rooms.
# But we can ensure we don't schedule too many big sessions at any given time:
sizes = sorted(room_sizes, reverse=True) # biggest first
for i, size in enumerate(sizes):
    if i+1 < len(sizes) and sizes[i] == sizes[i+1]:
        # Multiple rooms of the same size, wait for the last one
        continue
    for t in range(N_times):
        # At most 0 sessions larger than our largest room
        # At most 1 session larger than our second-largest room
        # At most N-1 sessions larger than our smallest room
        big_sess = [V[s,t] for s in range(N_sessions) if sess_interest[s] > size]
        if big_sess:
            p.at_most(i, big_sess)

# ans = p.minimize(0) # no objective, just find an answer
%time ans = p.minimize(conflicts, time_limit=time_limit_sec) # so everyone can see everything

ss, tt = np.vectorize(lambda x: ans['by_var'][x])(V).round().nonzero()
for t, d, s in sorted(zip(tt, sess_interest[ss], ss)):
    print(f'Time {t}    Session {s:2d}   Demand {d:3d}')
    # print(f'Time {t}    Room {r}    Session {s:2d}   Demand {sess_interest[s]:3d} of {room_sizes[r]:3d}')
print(f'Objective:    {ans['soln'].fun}')
print(f'Lower bound:  {ans['soln'].mip_dual_bound}    (gap = {ans['soln'].mip_gap})')
print(f'# variables:  {len(ans['vs'])}')
print(f'# constraints:  {len(ans['constraints'].A)}')
print(ans['soln'].message)
